In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageNet, CIFAR10

from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.UTILS_TORCH import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently using: '{device}'")

Currently using: 'cuda'


In [3]:
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARN_RATE = 0.001
ALPHA = 0.1
TEMP = 3
NUM_WORKERS = 16
OVERLAY_PROB = 0.25

In [4]:
IGS = "PyTorch_CIFAR10/Captum_IGs.npy"
print(f"Using IGs: {IGS}")

# Load the precomputed IGs
igs = np.load(IGS)
print(f"IGs shape: {igs.shape}")

Using IGs: PyTorch_CIFAR10/Captum_IGs.npy
IGs shape: (50000, 32, 32)


In [5]:
precomputed_logits = np.load("PyTorch_CIFAR10/data/cifar10_logits.npy")
print("Shape of teacher_logits:", precomputed_logits.shape)

Shape of teacher_logits: (50000, 10)


In [6]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

student_aug = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

In [7]:
train_dataset = CIFAR10WithIG(
    igs=igs,
    root="PyTorch_CIFAR10/data",
    train=True,
    transform=student_aug,
    precomputed_logits=precomputed_logits,
    overlay_prob=OVERLAY_PROB,
    return_ig=False,
)

# Load the data into batches
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

# load student data
test_data = CIFAR10(
    root="PyTorch_CIFAR10/data",
    train=False,
    transform=transform,
)

test_loader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

Files already downloaded and verified


In [8]:
Teacher = mobilenet_v2(pretrained=True)
Teacher.to(device);

In [9]:
teacher_params = count_parameters(Teacher)

In [10]:
Student = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student.to(device);

In [11]:
student_params = count_parameters(Student)

In [12]:
teacher_params / student_params

4.115345410654685

# First Student

In [13]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-3])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [14]:
Student_1 = Smaller(mobilenet_v2(pretrained=False))
Student_1.to(device);

In [15]:
S1 = count_parameters(Student_1)
teacher_params / S1

2.194111842040726

In [16]:
Student_1, acc_1 = train_eval_kd(
    student=Student_1,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_2.19_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.9024,            Accuracy: 38.43%


Epoch 2/100: Loss: 1.5642,            Accuracy: 55.56%


Epoch 3/100: Loss: 1.3814,            Accuracy: 64.31%


Epoch 4/100: Loss: 1.2654,            Accuracy: 69.83%


Epoch 5/100: Loss: 1.1822,            Accuracy: 73.75%


...

Epoch 96/100: Loss: 0.6089,            Accuracy: 97.48%


Epoch 97/100: Loss: 0.6098,            Accuracy: 97.31%


Epoch 98/100: Loss: 0.6080,            Accuracy: 97.38%


Epoch 99/100: Loss: 0.6081,            Accuracy: 97.45%


Epoch 100/100: Loss: 0.6041,            Accuracy: 97.60%


In [17]:
print(f"Test Acc = {acc_1:.2f}%")

Test Acc = 90.87%


# Second Student

In [18]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-7])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [19]:
Student_2 = Smaller(mobilenet_v2(pretrained=False))
Student_2.to(device);

In [20]:
S2 = count_parameters(Student_2)
teacher_params / S2

7.286700938902897

In [21]:
Student_2, acc_2 = train_eval_kd(
    student=Student_2,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_7.29_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.7537,            Accuracy: 45.98%


Epoch 2/100: Loss: 1.4343,            Accuracy: 62.16%


Epoch 3/100: Loss: 1.2799,            Accuracy: 69.39%


Epoch 4/100: Loss: 1.1879,            Accuracy: 73.82%


Epoch 5/100: Loss: 1.1244,            Accuracy: 76.94%


...

Epoch 96/100: Loss: 0.6384,            Accuracy: 96.85%


Epoch 97/100: Loss: 0.6357,            Accuracy: 97.09%


Epoch 98/100: Loss: 0.6338,            Accuracy: 97.03%


Epoch 99/100: Loss: 0.6359,            Accuracy: 96.94%


Epoch 100/100: Loss: 0.6324,            Accuracy: 97.05%


In [22]:
print(f"Test Acc = {acc_2:.2f}%")

Test Acc = 91.40%


# Third Student

In [23]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-9])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [24]:
Student_3 = Smaller(mobilenet_v2(pretrained=False))
Student_3.to(device);

In [25]:
S3 = count_parameters(Student_3)
teacher_params / S3

12.042134619733172

In [26]:
Student_3, acc_3 = train_eval_kd(
    student=Student_3,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_12.04_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.7743,            Accuracy: 45.09%


Epoch 2/100: Loss: 1.4812,            Accuracy: 60.15%


Epoch 3/100: Loss: 1.3405,            Accuracy: 67.10%


Epoch 4/100: Loss: 1.2430,            Accuracy: 71.87%


Epoch 5/100: Loss: 1.1780,            Accuracy: 74.95%


...

Epoch 96/100: Loss: 0.7023,            Accuracy: 95.82%


Epoch 97/100: Loss: 0.7045,            Accuracy: 95.72%


Epoch 98/100: Loss: 0.7026,            Accuracy: 95.81%


Epoch 99/100: Loss: 0.7036,            Accuracy: 95.67%


Epoch 100/100: Loss: 0.6992,            Accuracy: 95.87%


In [27]:
print(f"Test Acc = {acc_3:.2f}%")

Test Acc = 90.73%


# Fourth Student

In [28]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-11])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [29]:
Student_4 = Smaller(mobilenet_v2(pretrained=False))
Student_4.to(device);

In [30]:
S4 = count_parameters(Student_4)
teacher_params / S4

28.974816695598104

In [31]:
Student_4, acc_4 = train_eval_kd(
    student=Student_4,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_28.97_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.8009,            Accuracy: 43.63%


Epoch 2/100: Loss: 1.5178,            Accuracy: 58.18%


Epoch 3/100: Loss: 1.4093,            Accuracy: 63.97%


Epoch 4/100: Loss: 1.3370,            Accuracy: 67.51%


Epoch 5/100: Loss: 1.2820,            Accuracy: 70.57%


...

Epoch 96/100: Loss: 0.8443,            Accuracy: 91.13%


Epoch 97/100: Loss: 0.8463,            Accuracy: 91.01%


Epoch 98/100: Loss: 0.8396,            Accuracy: 91.33%


Epoch 99/100: Loss: 0.8416,            Accuracy: 91.22%


Epoch 100/100: Loss: 0.8402,            Accuracy: 91.35%


In [32]:
print(f"Test Acc = {acc_4:.2f}%")

Test Acc = 88.65%


# Fifth Student

In [33]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-13])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [34]:
Student_5 = Smaller(mobilenet_v2(pretrained=False))
Student_5.to(device);

S5 = count_parameters(Student_5)
teacher_params / S5

54.5931657310227

In [35]:
Student_5, acc_5 = train_eval_kd(
    student=Student_5,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_54.59_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.8100,            Accuracy: 43.38%


Epoch 2/100: Loss: 1.5776,            Accuracy: 55.59%


Epoch 3/100: Loss: 1.4736,            Accuracy: 61.11%


Epoch 4/100: Loss: 1.4119,            Accuracy: 64.21%


Epoch 5/100: Loss: 1.3681,            Accuracy: 66.54%


...

Epoch 96/100: Loss: 0.9679,            Accuracy: 86.07%


Epoch 97/100: Loss: 0.9678,            Accuracy: 86.13%


Epoch 98/100: Loss: 0.9655,            Accuracy: 86.26%


Epoch 99/100: Loss: 0.9667,            Accuracy: 86.18%


Epoch 100/100: Loss: 0.9645,            Accuracy: 86.13%


In [36]:
print(f"Test Acc = {acc_5:.2f}%")

Test Acc = 84.81%


# Sixth Student

In [37]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-15])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [38]:
Student_6 = Smaller(mobilenet_v2(pretrained=False))
Student_6.to(device);

S6 = count_parameters(Student_6)
teacher_params / S6

139.426630095998

In [39]:
Student_6, acc_6 = train_eval_kd(
    student=Student_6,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_139.43_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.8973,            Accuracy: 39.00%


Epoch 2/100: Loss: 1.6901,            Accuracy: 50.04%


Epoch 3/100: Loss: 1.6014,            Accuracy: 54.98%


Epoch 4/100: Loss: 1.5451,            Accuracy: 57.75%


Epoch 5/100: Loss: 1.5113,            Accuracy: 59.73%


...

Epoch 96/100: Loss: 1.1890,            Accuracy: 75.72%


Epoch 97/100: Loss: 1.1898,            Accuracy: 75.96%


Epoch 98/100: Loss: 1.1842,            Accuracy: 75.73%


Epoch 99/100: Loss: 1.1845,            Accuracy: 76.02%


Epoch 100/100: Loss: 1.1869,            Accuracy: 75.80%


In [40]:
print(f"Test Acc = {acc_6:.2f}%")

Test Acc = 76.05%


# Seventh Student

In [41]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-17])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [42]:
Student_7 = Smaller(mobilenet_v2(pretrained=True))
Student_7.to(device);

S7 = count_parameters(Student_7)
teacher_params / S7

1121.7061183550652

In [43]:
Student_7, acc_7 = train_eval_kd(
    student=Student_7,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_1121.71.csv",
)

Epoch 1/100: Loss: 2.1367,            Accuracy: 25.75%


Epoch 2/100: Loss: 1.9618,            Accuracy: 33.37%


Epoch 3/100: Loss: 1.9300,            Accuracy: 35.93%


Epoch 4/100: Loss: 1.9094,            Accuracy: 37.61%


Epoch 5/100: Loss: 1.8923,            Accuracy: 39.18%


...

Epoch 96/100: Loss: 1.6700,            Accuracy: 52.03%


Epoch 97/100: Loss: 1.6708,            Accuracy: 52.15%


Epoch 98/100: Loss: 1.6724,            Accuracy: 52.10%


Epoch 99/100: Loss: 1.6722,            Accuracy: 52.20%


Epoch 100/100: Loss: 1.6723,            Accuracy: 52.13%


In [44]:
print(f"Test Acc = {acc_7:.2f}%")

Test Acc = 51.53%


# Normal Student

In [45]:
Student_8 = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student_8.to(device)

S8 = count_parameters(Student_8)
teacher_params / S8

4.115345410654685

In [46]:
Student_8, acc_8 = train_eval_kd(
    student=Student_8,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_4.12_not_pretrained.csv",
)

Epoch 1/100: Loss: 1.7907,            Accuracy: 44.09%


Epoch 2/100: Loss: 1.4344,            Accuracy: 62.15%


Epoch 3/100: Loss: 1.2750,            Accuracy: 69.61%


Epoch 4/100: Loss: 1.1868,            Accuracy: 73.46%


Epoch 5/100: Loss: 1.1228,            Accuracy: 76.46%


...

Epoch 96/100: Loss: 0.6123,            Accuracy: 97.41%


Epoch 97/100: Loss: 0.6085,            Accuracy: 97.58%


Epoch 98/100: Loss: 0.6137,            Accuracy: 97.39%


Epoch 99/100: Loss: 0.6083,            Accuracy: 97.59%


Epoch 100/100: Loss: 0.6108,            Accuracy: 97.44%


In [47]:
print(f"Test Acc = {acc_8:.2f}%")

Test Acc = 92.05%
